In [2]:
from langchain_community.chat_models import ChatOpenAI


OPENAI_API_KEY = "hahaha" 

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY)

In [3]:
llm.invoke("how can langsmith help with testing?")


AIMessage(content='Langsmith can help with testing in the following ways:\n\n1. Test Automation: Langsmith can generate test scripts automatically, reducing the manual effort required for writing test cases. It can create a wide range of test scenarios and generate test data efficiently.\n\n2. Test Coverage: Langsmith can analyze the codebase and identify areas that are not adequately covered by tests. It can suggest additional test cases to improve the overall test coverage.\n\n3. Bug Detection: Langsmith can analyze the code and identify potential bugs or vulnerabilities. It can detect common coding mistakes, such as null pointer exceptions, resource leaks, or concurrency issues, helping to prevent them from occurring during the testing phase.\n\n4. Test Prioritization: Langsmith can prioritize the execution of test cases based on various factors, such as code changes, risk analysis, or critical functionality. It helps in optimizing the testing process by ensuring that the most criti

In [4]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

chain = prompt | llm | output_parser

chain.invoke({"input": "how can langsmith help with testing?"})

'Langsmith can be a valuable tool for testing in several ways:\n\n1. Test Case Generation: Langsmith can automatically generate test cases based on the specifications of your software. By providing it with a formal specification or requirements document, Langsmith can generate a suite of test cases that cover various scenarios and edge cases. This can help ensure thorough test coverage and reduce the manual effort required to create test cases.\n\n2. Test Data Generation: Generating realistic and diverse test data can be time-consuming and challenging. Langsmith can help by automatically generating test data that covers a wide range of input values, ensuring that your tests cover different scenarios and potential issues. This can help identify bugs and vulnerabilities that may go unnoticed with limited or unrealistic test data.\n\n3. Test Oracles: Langsmith can assist in defining test oracles, which are mechanisms to determine if the system under test behaves correctly or not. By speci

In [5]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")

docs = loader.load()

In [11]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')
from langchain_openai import OpenAIEmbeddings

#Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.',
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
#embeddings = model.encode(sentences)
embeddings = OpenAIEmbeddings()




In [13]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain.text_splitter import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = DocArrayInMemorySearch.from_documents(documents, embeddings)
print(vector)

In [14]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [ ]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

# LangSmith offers several features that can help with testing:...

In [16]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

# First we need a prompt that we can pass into an LLM to generate this search query

prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [ ]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [18]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [ ]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

In [20]:
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

In [25]:
from langchain_community.tools.tavily_search import TavilySearchResults
import os

os.environ["TAVILY_API_KEY"] = "suck it"
TAVILY_API_KEY = "suck it"

search = TavilySearchResults()

tools = [retriever_tool, search]

In [29]:
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
print("This is prompt:", prompt)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

This is prompt: input_variables=['agent_scratchpad', 'input'] input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')), MessagesPlaceholder(variable_name='chat_history', optional=True), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')), MessagesPlaceholder(varia

In [31]:
agent_executor.invoke({"input": "and now?"})



> Entering new AgentExecutor chain...
Now that you have access to the LangSmith search tool and the Tavily search engine, you can use them to find information and answer any questions you may have. 

If you have any specific queries or topics you would like to search for, please let me know and I will assist you in using the tools effectively.

> Finished chain.


{'input': 'and now?',
 'output': 'Now that you have access to the LangSmith search tool and the Tavily search engine, you can use them to find information and answer any questions you may have. \n\nIf you have any specific queries or topics you would like to search for, please let me know and I will assist you in using the tools effectively.'}

In [28]:
agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://abc7news.com/bay-area-rain-forecast-storm-timeline-california-weather/14260083/', 'content': "WATCH LIVE TIMELINE: Rain returns to Bay Area forecast with Sierra snow in 1st week of 2024  SAN FRANCISCO (KGO) -- After a dry New Year's, rain is back in the forecast for the first week of 2024. The Bay Area  2024. The Bay Area will see rain Tuesday night and an isolated shower on Wednesday morning. Thursday is a sunny and dry  is a sunny and dry day but showers return Friday evening and linger into the day Saturday.SAN FRANCISCO (KGO) -- After a dry New Year's, rain is back in the forecast for the first week of 2024. The Bay Area will see rain Tuesday night and an isolated shower on Wednesday morning ..."}, {'url': 'https://hoodline.com/2024/01/san-francisco-bay-area-faces-rapid-weather-shift-with-rain-winds-and-small-craft-advisories/', 'content':

{'input': 'what is the weather in SF?',
 'output': 'The weather in San Francisco is currently rainy. There will be rain on Tuesday night and an isolated shower on Wednesday morning. Thursday will be sunny and dry, but showers will return on Friday evening and linger into Saturday. You can find more information about the weather in San Francisco [here](https://abc7news.com/bay-area-rain-forecast-storm-timeline-california-weather/14260083/) and [here](https://hoodline.com/2024/01/san-francisco-bay-area-faces-rapid-weather-shift-with-rain-winds-and-small-craft-advisories/).'}